In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from scipy.stats import pmean

In [ ]:
taxonomic_levels = {"d": 0, "p": 1, "c": 2, "o": 3, "f": 4, "g": 5, "s": 6}

In [ ]:
def fix_index(df: pd.DataFrame, index_col: int) -> pd.DataFrame:
    if index_col == 0:
        df.set_index(df.columns[0], inplace=True)
        df.drop(columns=['even_stag', 'run', 'in_out', 'v_region'], inplace=True)
        return df

    else:
        df = df.iloc[:, index_col:]
        df.set_index(df.columns[0], inplace=True)
        return df

def determine_taxonomic_rank(split_classification: list, taxon_level: str) -> str:
    if len(split_classification) == taxonomic_levels[taxon_level]+1:
        return split_classification[-1]
    else:
        return "Assigned_Higher"
    
def clean_columns(df: pd.DataFrame, delimiter_pattern=";", transpose=False, taxon_level="g"):
    new_labels = []
    data = []
    if transpose:
        data = df.columns.to_list()
    else:
        data = df.index.to_list()

    for c in data:
        # print(c)
        pattern1 = re.compile(delimiter_pattern)
        col6 = re.split(pattern1, c)
        print(col6)

        org_name = determine_taxonomic_rank(col6, taxon_level)

        # pattern2 = re.compile(r'')
        # col6_clean = re.split(pattern2, col6)[-1]

        if org_name == '__':
            new_labels.append("Assigned_Higher")
            continue

        new_labels.append(org_name)

    # print(new_cols)
    if transpose:
        df.columns = new_labels
        display(df.head())
        return df 
    else:
        df.index = new_labels
        display(df.head())
        return df


def clean_csv(root, f, delimiter_pattern=";", transpose=False, index_col=0, taxon_level="g"):
    print(f)
    df = pd.read_csv(os.path.join(root, f))
    # display(df)

    df = fix_index(df, index_col)
    display(df.head())

    clean_columns(df, delimiter_pattern, transpose, taxon_level)

    if transpose:
        deduped = df.groupby(lambda x:x, axis=1).sum()
        return deduped.T

    else:
        deduped = df.groupby(lambda x:x, axis=0).sum()
        return deduped
        
cleaned_df = []

silva_dir = '/Volumes/TBHD/Bioinformatics/ion_torrent_qiime2_methods_manuscript/Classified_Tax_Counts_QIIME2_CSV_Files/Cutprimers/Species'
set_2_dir = '/Volumes/TBHD_share/multi_v_regions_tables'

for root, dirs, file in os.walk(set_2_dir):
    for f in file:
        # if "Silva" in f:
        #     cleaned_df.append(clean_csv(root, f))
        cleaned_df.append(clean_csv(root, f, delimiter_pattern=",", transpose=False, index_col=1, taxon_level="g"))

In [135]:
def extra_column_cleaning(df):
    cols = df.columns.to_list()
    new_cols = []
    for c in cols:
        pattern = re.compile(r'_V\d')
        splitted = re.split(pattern, c)
        new_cols.append(splitted[0])

    df.columns = new_cols
    return df

extra_clean = [extra_column_cleaning(df) for df in cleaned_df]
df1 = extra_clean[0].iloc[0:5, 0:8]
display(df1.head())
# df2 = extra_clean[1].iloc[0:5, 0:8]
# df3 = extra_clean[2].iloc[0:5, 0:8]

# print(df1.shape)
# print(df2.shape)
# print(len(extra_clean))
out = pd.concat(extra_clean, axis=0)
display(out)

groups = out.groupby(level=0, axis=0)
c = 0

final = pd.DataFrame()
for g in groups:
    # print(g[0])
    # display(g[1])
    m = pd.DataFrame(pmean(g[1], axis=0, p=1, nan_policy='omit')).T
    m.columns = g[1].columns
    m.index = [g[0]]

    final = pd.concat([final, m], axis=0)

    # display(m)
    c+=1
    # if c == 4:
        # break

display(final.head())
final.to_csv("final.csv")

,O-001-02,O-001-03,O-001-04,O-001-05,O-001-06,O-001-07,O-001-08,O-001-09
Assigned_Higher,1203,952,1244,969,756,684,855,842
g:Acinetobacter,0,2,0,1,0,0,0,0
g:Actinomyces,95,147,137,227,42,113,114,179
g:Aeriscardovia,0,0,0,0,0,0,0,0
g:Aerococcus,0,0,0,0,0,0,0,0


,O-001-02,O-001-03,O-001-04,O-001-05,O-001-06,O-001-07,O-001-08,O-001-09,O-001-10,O-002-01,...,O-023-08,O-023-09,O-023-10,O-004-10,O-005-01,O-005-06,O-005-07,O-005-08,O-005-09,O-022-03
Assigned_Higher,1203,952,1244,969,756,684,855,842,1145,1694,...,1054,1027,1409,NaN,NaN,NaN,NaN,NaN,NaN,NaN
g:Acinetobacter,0,2,0,1,0,0,0,0,1,0,...,2,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
g:Actinomyces,95,147,137,227,42,113,114,179,182,120,...,271,170,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
g:Aeriscardovia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
g:Aerococcus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
g:Subdoligranulum,0,0,0,0,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
g:Tannerella,2,0,0,0,0,0,0,0,0,0,...,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
g:Treponema,1,0,0,0,0,0,0,1,0,0,...,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
g:Veillonella,547,698,525,401,231,242,298,319,541,598,...,676,925,1174,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,O-001-02,O-001-03,O-001-04,O-001-05,O-001-06,O-001-07,O-001-08,O-001-09,O-001-10,O-002-01,...,O-023-08,O-023-09,O-023-10,O-004-10,O-005-01,O-005-06,O-005-07,O-005-08,O-005-09,O-022-03
Assigned_Higher,696.0,842.600000,869.800000,1010.800000,1081.2,872.8,895.6,839.2,690.800000,774.800000,...,840.600000,549.600000,539.400000,380.5,907.000000,1031.666667,1035.5,87.0,437.0,741.5
g:Abiotrophia,1.0,0.333333,0.333333,0.333333,0.0,0.0,0.0,0.0,1.333333,0.666667,...,2.333333,2.333333,2.333333,0.0,0.000000,0.000000,0.0,54.0,5.0,2.0
g:Acinetobacter,0.0,0.666667,0.000000,0.333333,0.0,0.0,0.0,0.0,0.333333,0.000000,...,0.666667,0.333333,0.000000,NaN,0.000000,1.000000,NaN,NaN,0.0,NaN
g:Actinobacillus,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,1.000000,3.000000,13.000000,0.0,0.000000,0.000000,0.0,NaN,NaN,0.0
g:Actinomyces,57.8,74.400000,63.600000,107.200000,44.8,58.0,55.4,78.8,70.600000,98.600000,...,121.200000,94.200000,44.000000,59.0,46.666667,72.333333,42.0,2.0,73.0,114.5
